# import library

In [ ]:
# Selenium 설치
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# # 밑 부분은 konlpy와 mecab 설치
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

     |████████████████████████████████| 979 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 52.1 MB/s 
     |████████████████████████████████| 359 kB 55.2 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 4.0 MB 45.5 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://developer.download.nvidi

In [ ]:
def get_maker_info(driver):
    num_total = 0
    num_succ = 0
    num_failed = 0
    # 진행중인 펀딩은 실패로 처리하면 안 되니까
    sections = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[2]/div/div/div[1]').find_elements(By.TAG_NAME, 'section')
    for section in sections:
        text = section.find_element(By.TAG_NAME, 'h2').text[:-1]
        if text =="진행 중 펀딩":
            while True:
                try:
                    button = section.find_element(By.XPATH, 'div[3]').find_element(By.CSS_SELECTOR, 'button')
                    webdriver.ActionChains(driver).click(button).perform()
                    time.sleep(1)
                except:
                    break
                finally:
                    products = section.find_element(By.XPATH, 'div[2]')
                    labels = products.find_elements(By.TAG_NAME, "span")
                    
            for idx, item in enumerate(labels):
                if "%" in item.text :
                    if int(item.text.replace("%",""))>=100 :
                        num_succ += 1
                        num_total += 1
                    else:
                        num_total += 1
        elif text =="종료된 펀딩":
            while True:
                try:
                    button = section.find_element(By.XPATH, 'div[3]').find_element(By.CSS_SELECTOR, 'button')
                    webdriver.ActionChains(driver).click(button).perform()
                    time.sleep(1)
                except:
                    break
                finally:
                    products = section.find_element(By.XPATH, 'div[2]')
                    labels = products.find_elements(By.TAG_NAME, "span")
            
            for idx, item in enumerate(labels):
                if "%" in item.text :
                    if int(item.text.replace("%",""))>=100 :
                        num_succ += 1
                        num_total += 1
                    else:
                        num_failed += 1
                        num_total += 1
            
    rate = None
    num_sup = 0
    num_div = len(driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]').find_elements(By.TAG_NAME, 'div'))
    if num_div == 4:
        rate = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]/div[1]/span[1]').text.split(" ")[1]
        num_sup = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]/div[3]').text.split(" ")[1].replace("명", "")
    elif num_div == 3:
        rate = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]/div[1]/span[1]').text.split(" ")[1]
        num_sup = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]/div[2]').text.split(" ")[1].replace("명", "")
    elif num_div == 2:
        num_sup = driver.find_element(By.XPATH, '//*[@id="maker-profile-app"]/div[1]/div[2]/div[2]/div[1]').text.split(" ")[1].replace("명", "")
    else:
        pass
    try:    
      success_rate = num_succ / num_total
    except:
      success_rate = 0
    return success_rate, rate, num_sup

In [ ]:
# 구글 드라이브 연동
from google.colab import drive 

drive.mount('/content/gdrive/')
download_path = '/content/gdrive/MyDrive/'
!ln -s /content/gdrive/MyDrive/ /mydrive

# import library
import time
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By 
import time
import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import re
from tqdm import tqdm

# from konlpy.tag import Okt
# import konlpy
# from konlpy.tag import Kkma, Komoran, Hannanum, Okt
# from konlpy.utils import pprint
# from konlpy.tag import Mecab


# Colab설정 + 빠르게, Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {'profile.default_content_setting_values': { 'images': 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome('chromedriver', options=options)

# 링크 정보 가져오기
with open("/mydrive/CapstonDesign/이가원/Links.txt", "r") as file:
  links = file.readlines()

new_links = []
for link in links:
  link = link.rstrip("\n")
  new_links.append(link)
new_links = new_links[::-1]
print(len(new_links))

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
ln: failed to create symbolic link '/mydrive/MyDrive': File exists


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


37885


# 제목, 본문 가져오기

In [ ]:
# 리워드 상세정보, 키워드 추출

offset = 0 # 다시 시작할 위치

# infos.txt 파일에 sub_category, 제목, 본문내용 가져오기
with open("infos_1.txt", "a") as f:
  for idx in tqdm(range(0, 20)): # 추출할 new_link 범위
      time.sleep(1)

      # url, code 가져오기
      url = new_links[idx]
      code = url.split('/')[-1]

      # html 가져오기, driver.get
      resp = requests.get(url)
      html = resp.text
      soup = BeautifulSoup(html, 'html.parser')

      # 반환정책 url
      url = f'https://www.wadiz.kr/web/campaign/detail/fundingInfo/{code}?_refer_section_st=REWARD_1' 
      driver.get(url)
      sub_category = driver.find_element(By.XPATH, '//*[@id="reward-static-product-content-app"]/div/div/p/span').text # 리워드 상세정보

      # 스토리 url
      url = f'https://www.wadiz.kr/web/campaign/detail/{code}?_refer_section_st=REWARD_1'
      driver.get(url)
      title = driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/h2/a').text # 제목
      title = re.sub('[^A-Za-z0-9가-힣]', ' ', title) # 특수문자 제거
      explain = driver.find_element(By.XPATH, '//*[@id="container"]/div[6]/div/div[1]/div[2]/div/div/section/div[3]').text # 제목 밑 설명란 내용
      explain = re.sub('[^A-Za-z0-9가-힣]', ' ', explain) # 특수문자 제거
      main = driver.find_element(By.XPATH, '//*[@id="introdetails"]/div[2]').text # 본문 내용
      main = re.sub('[^A-Za-z0-9가-힣]', ' ', main) # 특수문제 제거
      
      print(f'code : {code} / sub_category: {sub_category} / title : {title} / explain : {explain} / main : {main}')
      print(f'{code};{sub_category};{title};{explain};{main}', file=f)

  5%|▌         | 1/20 [00:10<03:10, 10.05s/it]

code : 145716 / sub_category: 기타 재화 / title :  캠핑  고급요리 편백나무찜을 캠핑가서  서랍형편백나무찜기 / explain : 서랍형 편백나무 찜기는 독특한 디자인  실용성  편리성 까지 모든 것에서 차별화한 캠핑 편백찜기입니다  고급 편백나무찜요리를 이제는 캠핑가서도 서랍형편백찜기로 즐겨보세요  / main : 저희는 디자인부터 상품제작 까지 모두 직접 하고있습니다  핸드메이드 제품이기에 안심하고 사용하셔도 좋습니다  또한 짜집기나무가 아닌 원목 그대로를 재단하여 사용하였기에 더 견고합니다  안심하고 저희 제품 이용해주세요 


 10%|█         | 2/20 [00:18<02:39,  8.85s/it]

code : 145312 / sub_category: 건강 기능 식품 / title :  월 1500원대  괴물용량 가성비 갑  5000IU 초고함량 비타민D / explain : 월 1 500원대 5000IU 초고함량 햇빛비타민D  기존 용량에서 업그레이드하여 6개월분  가성비까지 가득 담았습니다  100  사탕수수 친환경 패키지로 환경까지 생각합니다  / main : 6개월분 대용량  5000IU 초고함량 햇빛 비타민D  뼈건강에 이롭 비타민D 5000IU  와디즈 최초 런칭  ELOP의 초고함량 비타민D  뼈건강에 이롭 비타민D 5000IU 가 새롭게 돌아왔습니다  기존 3개월분에서 6개월분으로 용량은 UP  가격은 DOWN  와디즈에서 가장 먼저 선보입니다  품목제조신고증 및 제조사 GMP인증서 메이커 소개 안녕하세요   이롭코퍼레이션 대표 김새찬입니다  지난 펀딩  모두에게 이롭 6년근 홍삼스틱    장건강에 이롭 100억 유산균    눈과 혈행에 이롭 rTG오메가3 루테인 지아잔틴 에 이어 ELOP의 업그레이드 출시 제품  뼈건강에 이롭 비타민D 5000IU  대용량 제품을 가장 먼저 와디즈 서포터님께 보여드리게 되었습니다  우선 지난 펀딩 시 서포터님들께서 보내주신 많은 관심과 사랑에 진심을 담아 감사의 말씀을 전합니다  금번 펀팅 제품인  뼈건강에 이롭 비타민D 5000IU 는 외부 활동이 부족한 현대인이라면 꼭 섭취해야 하는 제품입니다   제 주위 사람들만 보더라도 건강 검진 시 대부분이 비타민D 결핍임을 확인할 수 있었고 이로 인해 비타민D가 현대인들에게 얼마나 부족하고 중요한지 직접적으로 확인할 수 있었습니다  이번 제품도 당연히 패키지는 100  사탕수수로 만들어진 친환경 패키지를 사용하였습니다   또한 기존 3개월분을 판매하며 단점을 보완해 섭취와 보관이 용이하도록 6개월분으로 업그레이드 출시하게 되었습니다   이렇듯 저희 이롭은 친환경 패키지  건강을 위한 다양한 제품 라인업까지 서포터 님들의 건강과 지구를 위해 언제나 노력하겠습니다 

 15%|█▌        | 3/20 [00:26<02:28,  8.73s/it]

...

code : 144789 / sub_category: 디지털 콘텐츠(음원, 게임, 인터넷강의 등) / title : 급이 다른 보고서를 만드는 엑셀 시각화 실무 패키지 / explain : No코딩  쉽고 빠른 엑셀로 차원이 다른 보고서 만들기  보기 좋은 데이터로 급이 다른 결과와 인사이트를 얻어보세요  입문자도 완강 후엔 본인이 만들어낸 보고서에 놀랄거에요  / main : 엑셀 데이터 시각화 실무 강의를 듣고 나면 데이터를 활용하기도 어려운데 엑셀의 시각화까지 만들기란 해보지 않은 사용자에겐 쉽지 않습니다  데이터를 이용해 내가 전달하려는 메시지가 무엇인지 목표만 정한다면 엑셀로도 멋진 시각화 자료를 만들 수 있습니다  비즈니스 데이터를 보지 않은 사람이 사용자가 만든 시각화 자료만으로도 커뮤니케이션이 가능하게 만들어주는 엑셀 데이터 시각화 실무 마스터 클래스 과정은 그만한 가치가 있는 과정입니다  강의 정보    본 강의는 일부 특수 직군이나 전문가만을 위한 어려운 도구 학습법에서 벗어나 엑셀이라는 모든 구성원이 쉽게 다룰 수 있는 분석도구로 데이터 시각화 능력을 수행할 수 있도록 합니다  데이터 시각화  왜 필요할까요   데이터 시각화 도구 시장 규모가 2021년 59억 달러에서 2026년에는 102억 달러로  연평균 11 6  성장할 전망입니다  이것은 디지털 전환을 맞이한 현 시대에 데이터 시각화가 얼마나 중요한지를 말해줍니다  회사의 가장 가치 있고 풍부한 자원 중 하나인 데이터를 이용해 만든 시각화는 의사결정에 영향을 미치게 됩니다  하지만 데이터 분석가나 데이터 사이언티스트가 아닌 실무자는 당장 엑셀과 워드 다루기도 어려워  다른 도구에 대한 접근성이 떨어지며 학습 기회도 부족하고  데이터 시각화는 포기하기 마련입니다   지루한 글과 숫자에서 벗어나 한눈에 들어오는 엑셀 데이터 시각화  엑셀의 가장 큰 장점은 다른 데이터와의 연동과 시각화 기능입니다  몇 번의 클릭만으로 그럴듯한 차트가 완성되어 직무 담당자가 보기 쉽게 만들어줍니다  비즈니스

 95%|█████████▌| 19/20 [02:42<00:08,  8.49s/it]

code : 144780 / sub_category: 가공식품 / title :  4차 펀딩  솜사탕 같은 부드러움  인생 수비드 항정살 / explain : 4차 펀딩  믿고 먹을 수 있는 부드러움  인생 수비드 항정살 스테이크  / main : 3차 펀딩까지 성공적으로 끝냈습니다  진심으로 감사합니다     항정살은 특유의 아삭 거리는 쫄깃함으로 우리 모두가 사랑하는 부위이죠   항정살은 천겹살이라고도 불리우죠  지방층이 천겹살이라고 불리울 만큼 굉장히 촘촘히 쌓여있는 부위인데 이 덕분에 꼬들꼬들한 식감을 넘어선 아삭 혹은 꼬득한 식감이 납니다  그 식감은 우리가 항정살을 사랑하는 가장 큰 이유중 하나죠    정말 다 좋은데 항정살의 이 장점이 가끔 단점이 됩니다  가끔 너무 질겨요   쫄깃함을 넘어선 아삭한 식감이 가끔씩 단점이 되어 너무 질길 때가 있습니다  마음한켠에 드는 우리의 바람이 있죠  항정살특유의 쫄깃아삭함을 유지한 채 부드러울 순 없을까   마치 김부장님의   김대리 박력있으면서 부드럽게 만들어봐  같은 소리겠지만 우린 이 불가사의한 문제를 해결했습니다     무려 5시간의 저온 수비드 조리로 탱글한데 부드러운 식감 완성   바로 모든 걸 부드럽게 만들어 주는 수비드 조리법을 선택했습니다  수비드의 특성상 재료의 수분을 충분히 머금게 하면서 조리하기에 항정살 재료의 탱글함을 유지할 수 있었고 장시간 조리로 부드러움을 만들 수 있었습니다    특제 마늘 시즈닝으로 감칠맛 UP   수비드로 식감만 개선한것이 아닙니다  수비드 조리를 할때 수비드 전용 비닐 백에 항정살을 넣어 진공후 조리를 하는데 그 과정에 특제 마늘 시즈닝과 같이 조리합니다   마늘시즈닝에서 오는 감칠맛은  수비드 항정살에 중독될것같은 맛을 만듭니다    통째로 조리해 탱글한 식감 UP   수비드 항정살은 기본적으로 통째로 조리했기에 항정살의 육즙을 가득 머금고 있습니다  그래서 꼭 수비드 항정살을 스테이크처럼 꼭 통째로 조리하신다음에 썰어서 드세요   육즙 폭발의 식감을 경험

100%|██████████| 20/20 [02:49<00:00,  8.48s/it]

code : 144712 / sub_category: 패션잡화 (모자,벨트,액세서리) / title : 빼기 위해 더한 안경   나사를 빼고 기술을 더하다 / explain : 판타스틱 플라스틱은 뻔하지 않으며  친환경적이고  합리적인 안경입니다  / main :  Fantastic Plastic    판타스틱 플라스틱 Fantastic   1  기막히게 좋은  환상적인  2  엄청난  굉장한 Hazel   Solid Black 모델 착용  판타스틱 플라스틱 의 탄생 스토리 Series  Fantastic Plastic 안경의 프로젝트 구상 기존의 단점을 보완한  No Screw 2 0  의 구현으로 향상된 내구성과 자연스러운 디자인 JORA에 인증 받은 식물 원료 친환경 플라스틱과 유독 물질 발생이 적은 스테인리스 공정 직접 개발  직접 디자인  직접 생산으로 합리적인 가격대 실현 먼저 위 3가지 스토리를 프로젝트 방향으로 설정 한 후  이해 관계가 일치하는 디자인 및 설계  인젝션  가공  생산 등 각 분야의 실력 있는 전문가 10여명이 모여  효과적인 협업을 통해 기존과 차별 된 특별한 안경을 구현 할 수 있었습니다  첫 번째 프로젝트   No Screw 2 0 No Screw     안경의 각 부품을 결합하기 위해 나사를 쓰지 않고 다른 기술로 연결하는 방식  대부분 특허 등록된 자체 기술을 독점하기에 높은 가격대를 형성하고 있다  멋진 말이지요  이번 프로젝트에 있어서 가장 큰 영감을 받은 문장입니다  전체적으로 멋진 문장이지만 우선 빼기 디자인이라는 발상에 매료되었으며  여기서 착안해 기존 안경에 있는 나사를 모두 뺀다면 뻔하지 않고 멋진 디자인이 나올 것이라는 생각을 하게 되었습니다   버튼이 잔뜩 달려있는 과거의 핸드폰 좌  버튼을 빼고 터치 스크린을 더한 스마트폰 우  유선케이블과 주변기기가 가득한 과거의 데스크탑 PC 좌  주변기기와 케이블을 빼고 휴대성을 더한 태블릿 PC 우  지금까지 있어온 과거의 안경들 좌  나사를 빼고 내구성과 디자인

# 리워드 종류 (리워드 개수), 최저금액, 목표금액 / 금액, 메이커 정보, 이미지 개수, 펀딩기간, 카테고리명, 달성률, 과목코드, 리워드 상세정보같은 정보들 가져오기

In [ ]:
# 리워드 종류 (리워드 개수), 최저금액, 목표금액 / 금액, 메이커 정보, 이미지 개수, 펀딩기간, 카테고리명, 달성률, 과목코드, 리워드 상세정보

offset = 0 # 다시 시작할 위치

with open("infos_2.txt", "a") as f:
  for idx in tqdm(range(0,  20)): # 뽑아올 범위
      time.sleep(1)

      url = new_links[idx]
      code = url.split('/')[-1] # 종목코드  

      resp = requests.get(url)
      html = resp.text
      soup = BeautifulSoup(html, 'html.parser')
      driver.get(url)

      # 달성률 
      try:
        result = driver.find_element(By.XPATH, '//*[@id="container"]/div[6]/div/div[1]/div[1]/div[1]/div[1]/p[3]/strong').text
        # //*[@id="container"]/div[6]/div/div[1]/div[1]/div[1]/div[1]/p[1]/strong
        # //*[@id="container"]/div[6]/div/div[1]/div[1]/div[1]/div[1]/p[3]/strong  
      except:
        continue

      # 리워드 종류
      num_reward = len(soup.find_all('div', attrs={'class':'top-info'}))

      # 최저금액
      prices = []
      for i in range(len(soup.find_all('div', attrs={'class':'top-info'}))):
        string = soup.find_all('div', attrs={'class':'top-info'})[i].dt.text
        price = re.sub('[\D]', '', string)
        prices.append(int(price))
      try:
        while 1:
          min_price = min(prices)
          if min_price <= 5000:
            prices.remove(min_price)
          else:
            break  
      except:
        min_price = 0
        
      # 목표금액
      try:
        target = int(re.sub('[\D]', '', soup.select('div > div > p > span')[2].text))
      except:
        target = int(re.sub('[\D]', '', soup.select('div > div > p > span')[0].text))

      # 본문 이미지 개수
      cnt_main_images = len(soup.find_all('img', attrs={'class':'fr-fic fr-dib'})) 

      # 표지 이미지 개수
      cnt_title_images = len(soup.find_all('img', attrs={'class':'img-responsive'})) 

      # 펀딩기간 
      start_date, end_date = soup.select('div > div > p > br > span')[0].text.split('-')[0], soup.select('div > div > p > br > span')[0].text.split('-')[1]

      # 카테고리명
      category = soup.find('p', attrs={'class':'title-info'}).text 
      
      print(f"code: {code} / num_reward : {num_reward} / min_price: {min_price} / target: {target} / cnt_main_images: {cnt_main_images} / \
cnt_title_images: {cnt_title_images} / start_date: {start_date} / end_date:{end_date} / category: {category} / \
result: {result}")

      print(f"{code};{num_reward};{min_price};{target};{cnt_main_images};{cnt_title_images};{start_date};{end_date};{category};{result}", file=f)
      

  5%|▌         | 1/20 [00:12<03:59, 12.61s/it]

code: 145716 / num_reward : 2 / min_price: 173000 / target: 500000 / cnt_main_images: 160 / cnt_title_images: 1 / start_date: 2022.04.26 / end_date:2022.05.04 / category: 홈·리빙 / result: 176


 10%|█         | 2/20 [00:19<02:42,  9.06s/it]

code: 145312 / num_reward : 3 / min_price: 11900 / target: 500000 / cnt_main_images: 5 / cnt_title_images: 2 / start_date: 2022.04.20 / end_date:2022.04.29 / category: 푸드 / result: 442


 15%|█▌        | 3/20 [00:25<02:12,  7.80s/it]

...

code: 144789 / num_reward : 4 / min_price: 79000 / target: 500000 / cnt_main_images: 19 / cnt_title_images: 1 / start_date: 2022.04.22 / end_date:2022.05.06 / category: 클래스·컨설팅 / result: 167


 95%|█████████▌| 19/20 [01:50<00:04,  4.91s/it]

code: 144780 / num_reward : 4 / min_price: 21800 / target: 500000 / cnt_main_images: 20 / cnt_title_images: 1 / start_date: 2022.04.06 / end_date:2022.04.22 / category: 푸드 / result: 244


100%|██████████| 20/20 [01:55<00:00,  5.80s/it]

code: 144712 / num_reward : 3 / min_price: 69000 / target: 500000 / cnt_main_images: 40 / cnt_title_images: 1 / start_date: 2022.04.20 / end_date:2022.05.11 / category: 패션·잡화 / result: 457


# 모든 정보 다 가져오기

In [ ]:
# 리워드 종류 (리워드 개수), 최저금액, 목표금액 / 금액, 메이커 정보, 이미지 개수, 펀딩기간, 카테고리명, 달성률, 과목코드, 리워드 상세정보
# 제목, 본문 설명란 가져오기 

offset = 0 # 다시 시작할 위치

# infos_1에는 제목, 본문, 설명란 등등
# infos_2에는 이미지개수같은 정보들

with open("infos_1.txt", "a") as f_1:
  with open("infos_2.txt", "a") as f_2:
    for idx in tqdm(range(0,  20)): # 뽑아올 범위
        time.sleep(1)

        url = new_links[idx]
        code = url.split('/')[-1] # 종목코드  

        resp = requests.get(url)
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')

        # 반환정책 url
        url = f'https://www.wadiz.kr/web/campaign/detail/fundingInfo/{code}?_refer_section_st=REWARD_1' 
        driver.get(url)
        sub_category = driver.find_element(By.XPATH, '//*[@id="reward-static-product-content-app"]/div/div/p/span').text # 리워드 상세정보

        # 스토리 url
        url = f'https://www.wadiz.kr/web/campaign/detail/{code}?_refer_section_st=REWARD_1'
        driver.get(url)
        title = driver.find_element(By.XPATH, '//*[@id="container"]/div[3]/h2/a').text # 제목
        title = re.sub('[^A-Za-z0-9가-힣]', ' ', title) # 특수문자 제거
        explain = driver.find_element(By.XPATH, '//*[@id="container"]/div[6]/div/div[1]/div[2]/div/div/section/div[3]').text # 제목 밑 설명란 내용
        explain = re.sub('[^A-Za-z0-9가-힣]', ' ', explain) # 특수문자 제거
        main = driver.find_element(By.XPATH, '//*[@id="introdetails"]/div[2]').text # 본문 내용
        main = re.sub('[^A-Za-z0-9가-힣]', ' ', main) # 특수문제 제거
        
        driver.get(url)
        # 달성률 
        try:
          result = driver.find_element(By.XPATH, '//*[@id="container"]/div[6]/div/div[1]/div[1]/div[1]/div[1]/p[3]/strong').text  
        except:
          continue

        # 리워드 종류
        num_reward = len(soup.find_all('div', attrs={'class':'top-info'}))

        # 최저금액
        prices = []
        for i in range(len(soup.find_all('div', attrs={'class':'top-info'}))):
          string = soup.find_all('div', attrs={'class':'top-info'})[i].dt.text
          price = re.sub('[\D]', '', string)
          prices.append(int(price))
        try:
          while 1:
            min_price = min(prices)
            if min_price <= 5000:
              prices.remove(min_price)
            else:
              break  
        except:
          min_price = 0
          
        # 목표금액
        try:
          target = int(re.sub('[\D]', '', soup.select('div > div > p > span')[2].text))
        except:
          target = int(re.sub('[\D]', '', soup.select('div > div > p > span')[0].text))

        # 본문 이미지 개수
        cnt_main_images = len(soup.find_all('img', attrs={'class':'fr-fic fr-dib'})) 

        # 표지 이미지 개수
        cnt_title_images = len(soup.find_all('img', attrs={'class':'img-responsive'})) 

        # 펀딩기간 
        start_date, end_date = soup.select('div > div > p > br > span')[0].text.split('-')[0], soup.select('div > div > p > br > span')[0].text.split('-')[1]

        # 카테고리명
        category = soup.find('p', attrs={'class':'title-info'}).text 
        
        # print(f'code : {code} / sub_category: {sub_category} / title : {title} / explain : {explain} / main : {main}')
        print(f'{code};{sub_category};{title};{explain};{main}', file=f_1)
  #       print(f"code: {code} / num_reward : {num_reward} / min_price: {min_price} / target: {target} / cnt_main_images: {cnt_main_images} / \
  # cnt_title_images: {cnt_title_images} / start_date: {start_date} / end_date:{end_date} / category: {category} / \
  # result: {result}")

        print(f"{code};{num_reward};{min_price};{target};{cnt_main_images};{cnt_title_images};{start_date};{end_date};{category};{result}", file=f_2)
      

  5%|▌         | 1/20 [00:13<04:17, 13.55s/it]

code : 145716 / sub_category: 기타 재화 / title :  캠핑  고급요리 편백나무찜을 캠핑가서  서랍형편백나무찜기 / explain : 서랍형 편백나무 찜기는 독특한 디자인  실용성  편리성 까지 모든 것에서 차별화한 캠핑 편백찜기입니다  고급 편백나무찜요리를 이제는 캠핑가서도 서랍형편백찜기로 즐겨보세요  / main : 저희는 디자인부터 상품제작 까지 모두 직접 하고있습니다  핸드메이드 제품이기에 안심하고 사용하셔도 좋습니다  또한 짜집기나무가 아닌 원목 그대로를 재단하여 사용하였기에 더 견고합니다  안심하고 저희 제품 이용해주세요 
code: 145716 / num_reward : 2 / min_price: 173000 / target: 500000 / cnt_main_images: 160 /   cnt_title_images: 1 / start_date: 2022.04.26 / end_date:2022.05.04 / category: 홈·리빙 /   result: 176


 10%|█         | 2/20 [00:25<03:44, 12.48s/it]

code : 145312 / sub_category: 건강 기능 식품 / title :  월 1500원대  괴물용량 가성비 갑  5000IU 초고함량 비타민D / explain : 월 1 500원대 5000IU 초고함량 햇빛비타민D  기존 용량에서 업그레이드하여 6개월분  가성비까지 가득 담았습니다  100  사탕수수 친환경 패키지로 환경까지 생각합니다  / main : 6개월분 대용량  5000IU 초고함량 햇빛 비타민D  뼈건강에 이롭 비타민D 5000IU  와디즈 최초 런칭  ELOP의 초고함량 비타민D  뼈건강에 이롭 비타민D 5000IU 가 새롭게 돌아왔습니다  기존 3개월분에서 6개월분으로 용량은 UP  가격은 DOWN  와디즈에서 가장 먼저 선보입니다  품목제조신고증 및 제조사 GMP인증서 메이커 소개 안녕하세요   이롭코퍼레이션 대표 김새찬입니다  지난 펀딩  모두에게 이롭 6년근 홍삼스틱    장건강에 이롭 100억 유산균    눈과 혈행에 이롭 rTG오메가3 루테인 지아잔틴 에 이어 ELOP의 업그레이드 출시 제품  뼈건강에 이롭 비타민D 5000IU  대용량 제품을 가장 먼저 와디즈 서포터님께 보여드리게 되었습니다  우선 지난 펀딩 시 서포터님들께서 보내주신 많은 관심과 사랑에 진심을 담아 감사의 말씀을 전합니다  금번 펀팅 제품인  뼈건강에 이롭 비타민D 5000IU 는 외부 활동이 부족한 현대인이라면 꼭 섭취해야 하는 제품입니다   제 주위 사람들만 보더라도 건강 검진 시 대부분이 비타민D 결핍임을 확인할 수 있었고 이로 인해 비타민D가 현대인들에게 얼마나 부족하고 중요한지 직접적으로 확인할 수 있었습니다  이번 제품도 당연히 패키지는 100  사탕수수로 만들어진 친환경 패키지를 사용하였습니다   또한 기존 3개월분을 판매하며 단점을 보완해 섭취와 보관이 용이하도록 6개월분으로 업그레이드 출시하게 되었습니다   이렇듯 저희 이롭은 친환경 패키지  건강을 위한 다양한 제품 라인업까지 서포터 님들의 건강과 지구를 위해 언제나 노력하겠습니다 

 15%|█▌        | 3/20 [00:36<03:22, 11.94s/it]

...

code : 144789 / sub_category: 디지털 콘텐츠(음원, 게임, 인터넷강의 등) / title : 급이 다른 보고서를 만드는 엑셀 시각화 실무 패키지 / explain : No코딩  쉽고 빠른 엑셀로 차원이 다른 보고서 만들기  보기 좋은 데이터로 급이 다른 결과와 인사이트를 얻어보세요  입문자도 완강 후엔 본인이 만들어낸 보고서에 놀랄거에요  / main : 엑셀 데이터 시각화 실무 강의를 듣고 나면 데이터를 활용하기도 어려운데 엑셀의 시각화까지 만들기란 해보지 않은 사용자에겐 쉽지 않습니다  데이터를 이용해 내가 전달하려는 메시지가 무엇인지 목표만 정한다면 엑셀로도 멋진 시각화 자료를 만들 수 있습니다  비즈니스 데이터를 보지 않은 사람이 사용자가 만든 시각화 자료만으로도 커뮤니케이션이 가능하게 만들어주는 엑셀 데이터 시각화 실무 마스터 클래스 과정은 그만한 가치가 있는 과정입니다  강의 정보    본 강의는 일부 특수 직군이나 전문가만을 위한 어려운 도구 학습법에서 벗어나 엑셀이라는 모든 구성원이 쉽게 다룰 수 있는 분석도구로 데이터 시각화 능력을 수행할 수 있도록 합니다  데이터 시각화  왜 필요할까요   데이터 시각화 도구 시장 규모가 2021년 59억 달러에서 2026년에는 102억 달러로  연평균 11 6  성장할 전망입니다  이것은 디지털 전환을 맞이한 현 시대에 데이터 시각화가 얼마나 중요한지를 말해줍니다  회사의 가장 가치 있고 풍부한 자원 중 하나인 데이터를 이용해 만든 시각화는 의사결정에 영향을 미치게 됩니다  하지만 데이터 분석가나 데이터 사이언티스트가 아닌 실무자는 당장 엑셀과 워드 다루기도 어려워  다른 도구에 대한 접근성이 떨어지며 학습 기회도 부족하고  데이터 시각화는 포기하기 마련입니다   지루한 글과 숫자에서 벗어나 한눈에 들어오는 엑셀 데이터 시각화  엑셀의 가장 큰 장점은 다른 데이터와의 연동과 시각화 기능입니다  몇 번의 클릭만으로 그럴듯한 차트가 완성되어 직무 담당자가 보기 쉽게 만들어줍니다  비즈니스

 95%|█████████▌| 19/20 [03:52<00:11, 11.80s/it]

code : 144780 / sub_category: 가공식품 / title :  4차 펀딩  솜사탕 같은 부드러움  인생 수비드 항정살 / explain : 4차 펀딩  믿고 먹을 수 있는 부드러움  인생 수비드 항정살 스테이크  / main : 3차 펀딩까지 성공적으로 끝냈습니다  진심으로 감사합니다     항정살은 특유의 아삭 거리는 쫄깃함으로 우리 모두가 사랑하는 부위이죠   항정살은 천겹살이라고도 불리우죠  지방층이 천겹살이라고 불리울 만큼 굉장히 촘촘히 쌓여있는 부위인데 이 덕분에 꼬들꼬들한 식감을 넘어선 아삭 혹은 꼬득한 식감이 납니다  그 식감은 우리가 항정살을 사랑하는 가장 큰 이유중 하나죠    정말 다 좋은데 항정살의 이 장점이 가끔 단점이 됩니다  가끔 너무 질겨요   쫄깃함을 넘어선 아삭한 식감이 가끔씩 단점이 되어 너무 질길 때가 있습니다  마음한켠에 드는 우리의 바람이 있죠  항정살특유의 쫄깃아삭함을 유지한 채 부드러울 순 없을까   마치 김부장님의   김대리 박력있으면서 부드럽게 만들어봐  같은 소리겠지만 우린 이 불가사의한 문제를 해결했습니다     무려 5시간의 저온 수비드 조리로 탱글한데 부드러운 식감 완성   바로 모든 걸 부드럽게 만들어 주는 수비드 조리법을 선택했습니다  수비드의 특성상 재료의 수분을 충분히 머금게 하면서 조리하기에 항정살 재료의 탱글함을 유지할 수 있었고 장시간 조리로 부드러움을 만들 수 있었습니다    특제 마늘 시즈닝으로 감칠맛 UP   수비드로 식감만 개선한것이 아닙니다  수비드 조리를 할때 수비드 전용 비닐 백에 항정살을 넣어 진공후 조리를 하는데 그 과정에 특제 마늘 시즈닝과 같이 조리합니다   마늘시즈닝에서 오는 감칠맛은  수비드 항정살에 중독될것같은 맛을 만듭니다    통째로 조리해 탱글한 식감 UP   수비드 항정살은 기본적으로 통째로 조리했기에 항정살의 육즙을 가득 머금고 있습니다  그래서 꼭 수비드 항정살을 스테이크처럼 꼭 통째로 조리하신다음에 썰어서 드세요   육즙 폭발의 식감을 경험

100%|██████████| 20/20 [04:02<00:00, 12.15s/it]

code : 144712 / sub_category: 패션잡화 (모자,벨트,액세서리) / title : 빼기 위해 더한 안경   나사를 빼고 기술을 더하다 / explain : 판타스틱 플라스틱은 뻔하지 않으며  친환경적이고  합리적인 안경입니다  / main :  Fantastic Plastic    판타스틱 플라스틱 Fantastic   1  기막히게 좋은  환상적인  2  엄청난  굉장한 Hazel   Solid Black 모델 착용  판타스틱 플라스틱 의 탄생 스토리 Series  Fantastic Plastic 안경의 프로젝트 구상 기존의 단점을 보완한  No Screw 2 0  의 구현으로 향상된 내구성과 자연스러운 디자인 JORA에 인증 받은 식물 원료 친환경 플라스틱과 유독 물질 발생이 적은 스테인리스 공정 직접 개발  직접 디자인  직접 생산으로 합리적인 가격대 실현 먼저 위 3가지 스토리를 프로젝트 방향으로 설정 한 후  이해 관계가 일치하는 디자인 및 설계  인젝션  가공  생산 등 각 분야의 실력 있는 전문가 10여명이 모여  효과적인 협업을 통해 기존과 차별 된 특별한 안경을 구현 할 수 있었습니다  첫 번째 프로젝트   No Screw 2 0 No Screw     안경의 각 부품을 결합하기 위해 나사를 쓰지 않고 다른 기술로 연결하는 방식  대부분 특허 등록된 자체 기술을 독점하기에 높은 가격대를 형성하고 있다  멋진 말이지요  이번 프로젝트에 있어서 가장 큰 영감을 받은 문장입니다  전체적으로 멋진 문장이지만 우선 빼기 디자인이라는 발상에 매료되었으며  여기서 착안해 기존 안경에 있는 나사를 모두 뺀다면 뻔하지 않고 멋진 디자인이 나올 것이라는 생각을 하게 되었습니다   버튼이 잔뜩 달려있는 과거의 핸드폰 좌  버튼을 빼고 터치 스크린을 더한 스마트폰 우  유선케이블과 주변기기가 가득한 과거의 데스크탑 PC 좌  주변기기와 케이블을 빼고 휴대성을 더한 태블릿 PC 우  지금까지 있어온 과거의 안경들 좌  나사를 빼고 내구성과 디자인